<h1 style="color: blue;">Notebook to analyze log files from multiple machines</h1>

<h2 style="color: green;">Install pyspark</h2>

In [3]:
pip install pyspark

<h2 style="color: green;">Load Spark libraries and start a unique session</h2>

In [5]:
import pyspark

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark=SparkSession.builder.config("spark.driver.host", "localhost").appName('DataMonitorAnalyzer').getOrCreate()

spark

<h2 style="color: green;">Load csv files from the common log directory</h2>

In [10]:
df_pyspark=spark.read.format('csv').load("C:/Users/dcbel/cybersecurity/monitoroutput", inferSchema=True, header=True)

<h2 style="color: green;">Show the first 20 rows of the initial dataframe</h2>

In [12]:
df_pyspark.show(20)

+-------------------+-------------+--------+------------+--------+-----------------+----------+--------------+------------+--------------------+------------+--------------------+-------------+------------+------------+---------+-----------------+--------------------+
|          Timestamp|Computer_Name|Total_MB|Available_MB| Used_MB|CPU_Usage_Percent|Bytes_Sent|Bytes_Received|File_Created|       File_Modified|File_Deleted|         Top_Process|Disk_Total_GB|Disk_Used_GB|Disk_Free_GB|Update_OS|Updates_Available|       Update_Titles|
+-------------------+-------------+--------+------------+--------+-----------------+----------+--------------+------------+--------------------+------------+--------------------+-------------+------------+------------+---------+-----------------+--------------------+
|2025-04-08 16:52:11|  Main_laptop|32529.69|    13014.23|19515.46|             44.6|      null|          null|        null|                null|        null|                null|        542.7|    

<h2 style="color: green;">Print the datatypes for each column</h2>

In [14]:
df_pyspark.printSchema()

root
 |-- Timestamp: string (nullable = true)
 |-- Computer_Name: string (nullable = true)
 |-- Total_MB: double (nullable = true)
 |-- Available_MB: double (nullable = true)
 |-- Used_MB: double (nullable = true)
 |-- CPU_Usage_Percent: double (nullable = true)
 |-- Bytes_Sent: integer (nullable = true)
 |-- Bytes_Received: integer (nullable = true)
 |-- File_Created: string (nullable = true)
 |-- File_Modified: string (nullable = true)
 |-- File_Deleted: string (nullable = true)
 |-- Top_Process: string (nullable = true)
 |-- Disk_Total_GB: double (nullable = true)
 |-- Disk_Used_GB: double (nullable = true)
 |-- Disk_Free_GB: double (nullable = true)
 |-- Update_OS: string (nullable = true)
 |-- Updates_Available: integer (nullable = true)
 |-- Update_Titles: string (nullable = true)



<h2 style="color: green;">Print the column names</h2>

In [16]:
df_pyspark.columns

['Timestamp',
 'Computer_Name',
 'Total_MB',
 'Available_MB',
 'Used_MB',
 'CPU_Usage_Percent',
 'Bytes_Sent',
 'Bytes_Received',
 'File_Created',
 'File_Modified',
 'File_Deleted',
 'Top_Process',
 'Disk_Total_GB',
 'Disk_Used_GB',
 'Disk_Free_GB',
 'Update_OS',
 'Updates_Available',
 'Update_Titles']

<h2 style="color: blue;">Create a new dataframe with the original update columns</h2>
<p style="color: red;">Add computer name, type of OS, how many updates are available and the title of the udate. Remove duplicates and null entries.</p>

In [18]:
df_updates = df_pyspark.select('Computer_Name', 'Update_OS', 'Updates_Available', 'Update_Titles')

<h2 style="color: green;"> Remove duplicates and null data</h2>

In [20]:
df_updates = df_updates.distinct().na.drop(how="any",thresh=2)

<h3 style="color: red;"> Write data to disk</h3>

In [22]:
df_updates.coalesce(1).write.option("header", True) \
                      .option("sep", ",") \
                      .mode("overwrite") \
                      .csv("C:/Users/dcbel/cybersecurity/output/updates")

<h2 style="color: green;">Show the new dataframe for easy analysis.</h2>

In [24]:
df_updates.show(truncate=False)

+---------------+---------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Computer_Name  |Update_OS|Updates_Available|Update_Titles                                                                                                                                                                                                                                                                                                                  |
+---------------+---------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<h2 style="color: red;">Generate 2 new columns that will have the usage percentage of RAM and disk</h2>

In [26]:
df_pyspark = df_pyspark.withColumn('RAM_Used_Percent', (df_pyspark['Used_MB']/df_pyspark['Total_MB'])*100)

In [27]:
df_pyspark = df_pyspark.withColumn('Disk_Used_Percent', (df_pyspark['Disk_Used_GB']/df_pyspark['Disk_Total_GB'])*100)

<h2 style="color: blue;">Create a new dataframe that will include:</h2>
    <ol>
       <li>Computer_Name</li> <li>Total_MB</li> <li>Used_MB</li> <li>RAM Used%</li> <li>CPU_Usage_Percent</li> <li>Bytes_Sent</li> <li>Bytes_Received</li> <li>Top_Process</li> <li>Disk Used %</li> </ol> 
    

In [29]:
df = df_pyspark.select('Computer_Name', 'RAM_Used_Percent', 'CPU_Usage_Percent', 'Bytes_Sent', 'Bytes_Received', 'Top_Process', 'Disk_Used_Percent')

In [30]:
df = df.orderBy(df.CPU_Usage_Percent.desc(),df.RAM_Used_Percent.desc())

<h3 style="color: red;"> Write data to disk</h3>

In [32]:
df.coalesce(1).write.option("header", True) \
                      .option("sep", ",") \
                      .mode("overwrite") \
                      .csv("C:/Users/dcbel/cybersecurity/output/systemwatch")

In [33]:
df.show()

+---------------+------------------+-----------------+----------+--------------+--------------------+------------------+
|  Computer_Name|  RAM_Used_Percent|CPU_Usage_Percent|Bytes_Sent|Bytes_Received|         Top_Process| Disk_Used_Percent|
+---------------+------------------+-----------------+----------+--------------+--------------------+------------------+
|DESKTOP-OF30OEI| 50.79550404568872|             97.3|    297145|      12050240|('OneDrive.exe', ...|59.791781831582824|
|DESKTOP-OF30OEI|51.437133277322964|             93.8|    420637|      12289641|('OneDrive.exe', ...|59.795467108899935|
| WIN_MIDDLEWARE|  79.2678011994581|             89.6|     10419|         79845|('Code.exe', 'Sys...| 53.91249861781726|
|DESKTOP-OF30OEI| 51.16396744020616|             86.0|    428035|      12744903|('WmiPrvSE.exe', ...| 59.87101529390087|
|        gandalf|  47.8417466629518|             85.7|     10318|         47634|('Xorg', 'file://...| 93.81054897739506|
| WIN_MIDDLEWARE| 80.26596626036

<h2 style="color: blue;"> Create new dataframe only with the file handler information</h2>

In [35]:
df_files = df_pyspark.select('Computer_Name', 'File_Created', 'File_Modified', 'File_Deleted')

<h2 style="color: green;">Order data by column files created</h2>

In [37]:
df_files = df_files.orderBy(df_files.File_Created.desc())

In [38]:
df_files = df_files.distinct().na.drop(how="any",thresh=1)

In [39]:
df_files.show(truncate=False)

+-------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------+------------+
|Computer_Name|File_Created|File_Modified                                                                                                                                  |File_Deleted|
+-------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------+------------+
|Main_laptop  |null        |C:\Program Files \Microsoft Visual Studio\2022\Community\Common7\IDE\Blend.exe                                                                 |null        |
|Main_laptop  |null        |C:\Program Files \Microsoft SQL Server\Client SDK\ODBC\170\Tools\Binn                                                                          |null        |
|Main_laptop  |null        |C:\Program Files \Oracle\VirtualBox\Virtua

<h3 style="color: red;">Write data to disk</h3>

In [41]:
df_files.coalesce(1).write.option("header", True) \
                      .option("sep", ",") \
                      .mode("overwrite") \
                      .csv("C:/Users/dcbel/cybersecurity/output/filewatcher")